In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import re
import requests as requests

from Bio import Entrez
from Bio import SeqIO

# Мини таск 1
### 1. 

In [2]:
df = pd.read_excel('UTexas Aptamer Database(1).xlsx')
df.columns = df.columns.str.strip()
df.columns = [col.lower().replace(',', '').replace('.', '').replace(" ", '_') for col in df.columns]
df.head()

,year_of_paper,link_to_pubmed_entry,journals,journal_doi,citation,type_of_nucleic_acid,name_of_aptamer,target,aptamer_sequence,sequence_length,...,ph,molecular_weight_of_target,application_as_quoted_in_the_referenced_paper,post-selex_modifications_to_the_aptamer,additional_information,serial_number,parent_sequence_serial_number,corresponding_author_name_email_address,please_fill_out_the_form_for_any_feedbacks/comments,aptagen_cross_referencing(check__aptamer_chemistry_affinity_length_gc_content_sequence)
0,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,CB-42,Cibacron Blue 3GA,5'GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAU...,132,...,7.6,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,The aptamer was reported in DNA (include thymi...,10000000,NaN,Szostak JW,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,B4-25,Reactive Blue 4,5'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUC...,133,...,7.6,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,DNA library/pool was used as a template to gen...,10000001,NaN,Szostak JW,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
2,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,wild type,T4 DNA polymerase (gp43),5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,...,NaN,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000002,NaN,Gold L,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
3,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,major variant,T4 DNA polymerase (gp43),5'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCC...,113,...,NaN,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000003,NaN,Gold L,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
4,1992,https://pubmed.ncbi.nlm.nih.gov/1741036/,Nature,https://doi.org/10.1038/355564a0,"Bock, L. C., Griffin, L. C., Latham, J. A., Ve...",ssDNA,15 mer (colloquially known as Bock DNA Aptamer),"Thrombin (Sigma), Human",5'GGTTGGTGTGGTTGG3',15,...,7.4,Not reported,"Diagnostic and therapeutic: ""We are at present...",Not applicable,Presumed minimized variant was found,10000004,NaN,Toole JJ,https://forms.gle/n4TzuyddXQrHYJXF9,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   year_of_paper                                                                            1495 non-null   int64  
 1   link_to_pubmed_entry                                                                     1495 non-null   object 
 2   journals                                                                                 1495 non-null   object 
 3   journal_doi                                                                              1495 non-null   object 
 4   citation                                                                                 1495 non-null   object 
 5   type_of_nucleic_acid                                          

In [4]:
df.isna().sum()

year_of_paper                                                                                 0
link_to_pubmed_entry                                                                          0
journals                                                                                      0
journal_doi                                                                                   0
citation                                                                                      0
type_of_nucleic_acid                                                                          0
name_of_aptamer                                                                               0
target                                                                                        0
aptamer_sequence                                                                              0
sequence_length                                                                               0
gc_content                              

In [5]:
df = df.drop('please_fill_out_the_form_for_any_feedbacks/comments', axis=1)

In [6]:
df['pool_random_region'].unique()

array([98, 97, 9, 60, 32, 30, 10, 15, nan, 72, 20, 40, 74, 120, 35, 76,
       71, 25, 14, 12, 80, 70, 50, 3, 22, 47, 21, 51, 59, 61, 26, 49, 41,
       52, 58, 64, 16, 34, 24, 36, 56, 45, 55, 39, 23, 53, 33, 43, 19, 38,
       31, 42, 44, 0, 62, 'TBD'], dtype=object)

In [7]:
df['pool_random_region'].replace('TBD', np.nan, inplace=True)

In [8]:
df['divalent_salt'] = df['divalent_salt'].str.strip()

In [9]:
def to_numeric(column):
    return pd.to_numeric(df[column], errors='coerce')

df['kd_(nm)'] = to_numeric('kd_(nm)')
df['sequence_length'] = to_numeric('sequence_length')
df['gc_content'] = to_numeric('gc_content')
df['pool_random_region'] = to_numeric('pool_random_region')

In [10]:
df.iloc[406, df.columns.get_loc('kd_(nm)')] = 100000
df.iloc[407, df.columns.get_loc('kd_(nm)')] = 10000000
df.iloc[1480, df.columns.get_loc('kd_(nm)')] = 23
df.iloc[441, df.columns.get_loc('kd_(nm)')] = 1000

In [11]:
for i in range(len(df["affinity"])):
    item = df["affinity"][i]
    if not re.match(r"Kd[:><]?\s(~)?[0-9.]+\s((±\s[0-9.]+)?)|(E-[0-9.]+)?\s[pnuμ]((M)|(mol/L))", item) and "Not reported" not in item:
        print(i, item)

0 Kd~600µM
1 Kd<100µM
4 Kd: 25-200 nM
5 Kd: 25-200 nM
13 Kd: exhibits biphasic binding, 0.51 ± 0.13 nM, 60 ± 52 nM
15 Kd: exhibits biphasic binding, 0.19 ± 0.02 nM, 49 ± 26 nM
16 Kd: exhibits biphasic binding, 0.13 ± 0.03 nM, 510 ± 30 nM
17 Kd: exhibits biphasic binding, 0.32 ± 0.07 nM, 140 ± 80 nM
19 Kd: 2.0*
22 Kd: 2.1*
24 Kd: 1.7*
25 Kd: 4.3*
26 Kd: 2.4*
27 Kd: 3.6*
29 Kd: 3.1*
30 Kd: 0.4*
31 Kd: 7.7, 9.5, 9.6*
32 Kd: 4.2*
33 Kd: 1.9*
34 Kd: 3.6*
35 Kd: 1.4*
36 Kd: 4.1*
37 Kd: 3.9*
38 Kd: 1.9*
39 Kd: 2.6*
40 Kd: 1.0*
41 Kd: 2.0*
42 Kd: 3.2*
43 Kd: 0.3*
44 Kd: 1.4*
45 Kd: 6.4, 6.5, 7.4, 9.7*
46 Kd: 2.3*
47 Kd: 7.4*
48 Kd: 0.7*
49 Kd: 0.2*
50 Kd: 4.6*
51 Kd: 2.4*
52 Kd: 3.9*
53 Kd: 0.5*
58 IC50: 3-5uM
59 IC50: 3-5uM
60 IC50: 3-5uM
72 Kd: 62-68 µM
73 Kd: 56-76 µM
76 Kd1: 0.20 ± 0.02 nM
Kd2: 42 ± 30 nM

77 Kd1: 0.42 ± 0.04 nM
Kd2: 182 ±94 nM

78 Kd1: 0.48 ± 0.04 nM
Kd2: 82 ± 23 nM
79 Kd1: 1.1 ±0.2 nM
Kd2: 180 ± 160 nM
80 Kd1: 20 ± 1 nM

81 Kd1:1.8 ±0.4 nM
Kd2: 31 ± 10 nM

82 Kd1: 1.4 ±0

В строке aptamer_sequence исправим опечатки

In [12]:
df['aptamer_sequence'] = df['aptamer_sequence'].str.replace("5'", '').str.strip("'3")
df['aptamer_sequence'].unique()

array(['GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAUGUUAUUGGGUGGGGGAAACUUAAGAAAACUACCACCCUUCAACAUUACCGCCCUUCAGCCUGCCAGCGCCCUGCAGCCCGGGAAGCUU',
       'GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUCCGAAAAAUCCGUACCCAACAUAGAACCCCCCCAGCGCUCACACGGACGCCCCAUUACGGCUAACCGAACGCCUGCAGCCCGGGAAGCUU',
       'GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUUAGUUUUAUAGCCCAAUAACUCAGGCUCUUGAUUGGUUUUCAAUAGAGAUAUAAAAUUCUUUUCAUAG',
       ..., 'GGACGGAUUGCGGCCGUUGUCUGUGGCGUCCGUUCG',
       'GAATTCCGCGTGTGCACACGCTCACAGTTACTATCGCTACGTTCTTTGGTAGTCCGTTCGGGAT',
       'GCGTGTGCACACGGTCACTTAGTATCGCTACGTTCTTTGGTTCCGTTCGG'], dtype=object)

### 3. 

In [13]:
Entrez.email = "arsendoronin@gmail.com"

In [14]:
last=''
lastseq=''

def fetch_protein_sequence(protein_name):
    global last
    global lastseq

    if last==protein_name:
        # print('повторка')
        return lastseq
    last=protein_name

    handle = Entrez.esearch(db="protein", term=protein_name, retmax=1)
    record = Entrez.read(handle)

    if record['IdList']:
        protein_id = record['IdList'][0]
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        seq_record = SeqIO.read(handle, "fasta")
        print(protein_name, ' найден, ура-ура')
        
        lastseq=seq_record
        
        handle.close()
        return seq_record.seq
    else:
        print(protein_name, ' не найден')
        lastseq=None
        return None

df['sequence_of_the_target_protein'] = df['target'].apply(fetch_protein_sequence)

Cibacron Blue 3GA  не найден
Reactive Blue 4  найден, ура-ура
T4 DNA polymerase (gp43)  найден, ура-ура
Thrombin (Sigma), Human  найден, ура-ура
Human imunnodeficiency virus type 1 reverse transcriptase (HIV-1-RT)  найден, ура-ура
Basic fibroblast growth factor (bFGF)  найден, ура-ура
Rev protein of HIV-1 (minimal Rev-binding element (RBE) found within the Rev Responsive Element (RRE))  не найден
α-Thrombin, Human  найден, ура-ура
Alpha thrombin (α-thrombin), Human  найден, ура-ура
Cyanocobalamin (vitamin B12)  найден, ура-ура
Cobinamide dicyanide  найден, ура-ура
Cyanocobalamin (vitamin B12)  найден, ура-ура
Cobinamide dicyanide  найден, ура-ура
Thrombin, Human  найден, ура-ура
Bronchodilator theophylline  не найден
L-citrulline [L-(+)-2-amino-5-ureidovaleric acid]  найден, ура-ура
L-arginine  найден, ура-ура
Protein kinase C beta II (protein kinase C βII)  найден, ура-ура
Vascular Endothelial Growth Factor (VEGF)  найден, ура-ура
Adenosine and Adenosine Triphosphate (ATP)  найден, ур

Variant surface glycoprotein 221 (sVSG 221)  не найден
Variant surface glycoprotein 117 (sVSG 117)  не найден
Mammalian translation initiation factor 4A (eIF4A)  найден, ура-ура
Differentiated pheochromocytoma cells (PC12 cells), Rat  найден, ура-ура
Glioblastoma-derived cell line (U251)  не найден
ATP (Adenosine triphosphate)  найден, ура-ура
D‐staphylococcal enterotoxin B peptide (respective SEB peptides)  не найден
D‐staphylococcal enterotoxin B peptide ( full-length SEB protein)  не найден
Antibody-mediated inhibition of cytotoxic T cell antigen-4 (CTLA-4)  не найден
CD28 receptor on T cell  найден, ура-ура
ATP (Adenosine triphosphate)  найден, ура-ура
MAb198 antibody, Rat  не найден
Pro‐urokinase (PLAU), Human  найден, ура-ура
4,4′-methylenedianiline (MDA)  найден, ура-ура
Sialyllactose  найден, ура-ура
Nuclear factor of activated T cells DNA binding domain (NFAT DBD)  не найден
C-terminal ribonuclease domain of colicin E3 (CRD of colicin E3)  не найден
Escherichia coli (E. coli) 

Receptor-binding domain (RBD) of vascular endothelial growth factor 165 (VEGF165)  не найден
Receptor-binding domain (RBD) of vascular endothelial growth factor 121 (VEGF121)  не найден
Prostate specific antigen (PSA)  найден, ура-ура
Zinc-finger antiviral protein (ZAP)  найден, ура-ура
Mouse transferrin receptor (TfR-ECD)  не найден
L-tryptophan  найден, ура-ура
(R)-ibuprofen  найден, ура-ура
(S)-ibuprofen  найден, ура-ура
Egg white lysozyme  найден, ура-ура
Chloramphenicol (Cam)  найден, ура-ура
Interleukin-17A (IL‐17A)  найден, ура-ура
Anti-FLAG M2 antibody  найден, ура-ура
Fc-gamma receptor III (FcγRIII) (CD16α)  не найден
C-Met receptor  найден, ура-ура
Escherichia coli (E. coli) O157:H7  найден, ура-ура
Human papillomavirus type 16 (HPV-16) E7 oncoprotein  найден, ура-ура
SARS CoV N protein  найден, ура-ура
Group A Streptococcus (GAS)*  найден, ура-ура
Human immunodeficiency virus (HIV)-1Bal gp120 protein  не найден
Bisphenol A (BPA)  найден, ура-ура
6 fluorine atoms difference b

Escherichia coli (E. coli) ATCC 25922  найден, ура-ура
Zika non-structural 1 (NS1) protein  найден, ура-ура
Cytotoxic T lymphocyte antigen-4 (CTLA-4)  найден, ура-ура
Bifidobacterium bifidum (B. bifidum)  найден, ура-ура
Ractopamine (RAC)  не найден
Escherichia coli (E. coli) O157:H7  найден, ура-ура
Glucagon receptor (GCGR)  найден, ура-ура
Osteoponin (OPN), Human  найден, ура-ура
Benzylpenicillin (penicillin G)  найден, ура-ура
Forkhead box protein M1 (FOXMI) DNA-binding domain (DBD)  не найден
Metastatic colorectal carcinoma cell line LoVo  не найден
Salmonella enteritidis (S. enteritidis)  найден, ура-ура
Shigella sonnei (S. sonnei)  найден, ура-ура
Gremlin-1  найден, ура-ура
Alpha-amanitin (α-amanitin) from Amanita phalloides (A. phalloides)  найден, ура-ура
Epithelial cell adhesion molecule (EpCAM)  найден, ура-ура
Antigen 85 (Ag85A) (FbpA)  найден, ура-ура
Staphylococcus aureus (S. aureus) surface molecules  найден, ура-ура
Mammaglobin B (MGB2)  найден, ура-ура
Mammaglobin A (MG

The fluorophore in Green fluorescent protein (GFP) (Ser65-Tyr66-Gly67)  не найден
RNA Mimic of Green Fluorescent Protein (Fluorophores, coupled with fluorescence-activated cell sorting (FACS); Escherichia coli  не найден
Coagulation Factor IXa (FIXa)  найден, ура-ура
D-arginine  найден, ура-ура
L-arginine  найден, ура-ура
Tenascin-C (TN-C)   найден, ура-ура
Prostate-Specific Membrane Antigen (PSMA) of prostate cancer cells  найден, ура-ура
Prostate-Specific Membrane Antigen (PSMA)  найден, ура-ура
Prostate-Specific Membrane Antigen(PSMA) of prostate cancer cells  найден, ура-ура
Recombinant Prostate-Specific Membrane Antigen (PSMA) of prostate cancer cells  не найден
Prostate-Specific Membrane Antigen (PSMA) of prostate cancer cells  найден, ура-ура
Recombinant Prostate-Specific Membrane Antigen (PSMA) of prostate cancer cells  не найден
Human T-cell acute lymphoblastic leukemia CCRF-CEM cells (later identified to bind PTK-7)  не найден
Prostate-Specific Membrane Antigen (PSMA)  найден

In [16]:
smiles = []
for i, seq in enumerate(df['target']):
    
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{seq}/JSON"
    response = requests.get(url)
    
    try:
        data = response.json()
        compound = data['PC_Compounds'][0]['props'][18]['value']['sval']
        smiles.append(compound)
        print(compound)
    except Exception:
        smiles.append(None)

C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=C(C=C4)NC5=NC(=NC(=N5)NC6=CC=CC=C6S(=O)(=O)O)Cl)S(=O)(=O)O)S(=O)(=O)O)N
C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=C(C=C4)S(=O)(=O)O)NC5=NC(=NC(=N5)Cl)Cl)S(=O)(=O)O)N
CC1=CC2=C(C=C1C)N(C=N2)[C@@H]3[C@@H]([C@@H]([C@H](O3)CO)OP(=O)([O-])O[C@H](C)CNC(=O)CC[C@@]4([C@H]([C@@H]5[C@]6([C@@]([C@@H](C(=N6)C(=C7[C@@]([C@@H](C(=N7)C=C8C([C@@H](C(=N8)C(=C4[N-]5)C)CCC(=O)N)(C)C)CCC(=O)N)(C)CC(=O)N)C)CCC(=O)N)(C)CC(=O)N)C)CC(=O)N)C)O.[C-]#N.[Co+3]
C/C/1=C/2\[C@@]([C@@H](C(=N2)/C=C\3/C([C@@H](C(=N3)/C(=C\4/[C@]([C@H](C([N-]4)[C@]5([C@@]([C@@H](C1=N5)CCC(=O)N)(C)CC(=O)N)C)CC(=O)N)(C)CCC(=O)NCC(C)O)/C)CCC(=O)N)(C)C)CCC(=O)N)(C)CC(=O)N.[C-]#N.[C-]#N.[Co]
C/C/1=C/2\[C@@]([C@@H](C(=N2)/C=C\3/C([C@@H](C(=N3)/C(=C\4/[C@]([C@H](C([N-]4)[C@]5([C@@]([C@@H](C1=N5)CCC(=O)N)(C)CC(=O)N)C)CC(=O)N)(C)CCC(=O)NCC(C)O)/C)CCC(=O)N)(C)C)CCC(=O)N)(C)CC(=O)N.[C-]#N.[C-]#N.[Co]
CC1=CC2=C(C=C1C)N(C=N2)[C@@H]3[C@@H]([C@@H]([C@H](O3)CO)OP(=O)([O-])O[C@H](C)CNC(=O)CC[C@@]4([C@H](

In [21]:
df['smiles'] = smiles

In [48]:
df.head()

,year_of_paper,link_to_pubmed_entry,journals,journal_doi,citation,type_of_nucleic_acid,name_of_aptamer,target,aptamer_sequence,sequence_length,...,molecular_weight_of_target,application_as_quoted_in_the_referenced_paper,post-selex_modifications_to_the_aptamer,additional_information,serial_number,parent_sequence_serial_number,corresponding_author_name_email_address,aptagen_cross_referencing(check__aptamer_chemistry_affinity_length_gc_content_sequence),smiles,new_sequence_of_the_target_protein
0,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,CB-42,Cibacron Blue 3GA,GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAUGU...,132,...,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,The aptamer was reported in DNA (include thymi...,10000000,NaN,Szostak JW,NaN,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
1,1990,https://pubmed.ncbi.nlm.nih.gov/1697402/,Nature,https://doi.org/10.1038/346818a0,"Ellington, A. D., & Szostak, J. W. (1990). In ...",ssRNA,B4-25,Reactive Blue 4,GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUCCG...,133,...,Not reported,"Detection: "" Isolate RNAs that bind to several...",Not applicable,DNA library/pool was used as a template to gen...,10000001,NaN,Szostak JW,NaN,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
2,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,wild type,T4 DNA polymerase (gp43),GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,...,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000002,NaN,Gold L,NaN,None,NaN
3,1990,https://pubmed.ncbi.nlm.nih.gov/2200121/,Science,https://doi.org/10.1126/science.2200121,"Tuerk, C., & Gold, L. (1990). Systematic evolu...",ssRNA,major variant,T4 DNA polymerase (gp43),GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,...,Not reported,"Detection: "" We have previously shown that the...",Not applicable,DNA library/pool was used as a template to gen...,10000003,NaN,Gold L,NaN,None,MKLSKDTTALLKNFATINSGIMLKSGQFIMTRAVNGTTYAEANISD...
4,1992,https://pubmed.ncbi.nlm.nih.gov/1741036/,Nature,https://doi.org/10.1038/355564a0,"Bock, L. C., Griffin, L. C., Latham, J. A., Ve...",ssDNA,15 mer (colloquially known as Bock DNA Aptamer),"Thrombin (Sigma), Human",GGTTGGTGTGGTTGG,15,...,Not reported,"Diagnostic and therapeutic: ""We are at present...",Not applicable,Presumed minimized variant was found,10000004,NaN,Toole JJ,NaN,None,NaN


In [47]:
df.isna().sum()

year_of_paper                                                                                 0
link_to_pubmed_entry                                                                          0
journals                                                                                      0
journal_doi                                                                                   0
citation                                                                                      0
type_of_nucleic_acid                                                                          0
name_of_aptamer                                                                               0
target                                                                                        0
aptamer_sequence                                                                              0
sequence_length                                                                               0
gc_content                              

In [51]:
df.drop(columns=[
    'year_of_paper',
    'link_to_pubmed_entry',
    'journals',
    'journal_doi',
    'citation',
    'name_of_aptamer',
    'target',
    'molecular_weight_of_target',
    'application_as_quoted_in_the_referenced_paper',
    'serial_number',
    'parent_sequence_serial_number',
    'corresponding_author_name_email_address',
    'aptagen_cross_referencing(check__aptamer_chemistry_affinity_length_gc_content_sequence'
], axis=1, inplace=True, errors='ignore')

In [53]:
df.isna().sum()

type_of_nucleic_acid                                                                          0
aptamer_sequence                                                                              0
sequence_length                                                                               0
gc_content                                                                                    0
affinity                                                                                      0
kd_(nm)                                                                                     380
pool_type                                                                                     0
pool_random_region                                                                           12
binding_buffer/conditions                                                                     0
divalent_salt                                                                               539
type_of_the_buffer                      

In [56]:
df = df.drop('aptagen_cross_referencing(check__aptamer_chemistry_affinity_length_gc_content_sequence)',axis=1)

In [57]:
df

,type_of_nucleic_acid,aptamer_sequence,sequence_length,gc_content,affinity,kd_(nm),pool_type,pool_random_region,binding_buffer/conditions,divalent_salt,type_of_the_buffer,ph,post-selex_modifications_to_the_aptamer,additional_information,smiles,new_sequence_of_the_target_protein
0,ssRNA,GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAUGU...,132,0.560606,Kd~600µM,600000.00,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98.0,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,Not applicable,The aptamer was reported in DNA (include thymi...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
1,ssRNA,GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUCCG...,133,0.593985,Kd<100µM,100000.00,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97.0,"0.6 ml of 0.5 M LiCI, 20 mM Tris-HCI, pH 7.6, ...",MgCl,Tris Buffers,7.6,Not applicable,DNA library/pool was used as a template to gen...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
2,ssRNA,GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,0.336283,Kd: 4.8 nM,4.80,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9.0,Not reported,NaN,Not Reported,NaN,Not applicable,DNA library/pool was used as a template to gen...,None,NaN
3,ssRNA,GAAUUGUGGUGUUGGCUCCCUAUAGUGAGUCGUAUUAAUAUUCCUU...,113,0.353982,Kd: 4.8 nM,4.80,5'-GAATTGTGGTGTTGGCTCCCTATAGTGAGTCGTATTA-ATATT...,9.0,Not reported,NaN,Not Reported,NaN,Not applicable,DNA library/pool was used as a template to gen...,None,MKLSKDTTALLKNFATINSGIMLKSGQFIMTRAVNGTTYAEANISD...
4,ssDNA,GGTTGGTGTGGTTGG,15,0.600000,Kd: 25-200 nM,112.50,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60.0,"19 mM Tris-acetate, pH 7.4, 140 mM NaCI, 5 mM ...",MgCl,Tris Buffers,7.4,Not applicable,Presumed minimized variant was found,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,5-uracil-modified-DNA,5’GCCTGUAUCCGCAGUAUCGGCAUUCAGCGAC3’\n\n\n,31,0.580645,Kd: 2.85 nM via filter binding assay,2.85,5′-TATGAGTGACCGTCCGCCTG-N40-CAGCCACACCACCAGCCA...,40.0,"40 mM HEPES (pH 7.5), 102 mM NaCl, 5 mM KCl, 5...",MgCl,Other Buffers,7.5,This is the truncated form of the full length ...,Modified nucleobase 5-[N-(1-naphthylmethyl)car...,None,NaN
1491,ssDNA,TTTTTAATTAAAGCTCGCCATCAAATAGCTTT,32,0.281250,"Kd: ""ΔPSap4#5 showed the highest PSA-binding a...",NaN,5′-CATGCTTACCTATAGTGAACttt-N24-tttCTTTGAGAACTG...,24.0,"TBS buffer (10 mM Tris/HCl, 150 mM NaCl, 5 mM ...",MgCl,Tris Buffers,7.4,This is a truncated fourth generation sequence...,NaN,None,NaN
1492,ssRNA,GGACGGAUUGCGGCCGUUGUCUGUGGCGUCCGUUCG,36,0.666667,Not reported,NaN,5'-TAATACGACTCACTATAGGGCGAATTCCGCGTGTGC-49N-GT...,49.0,DNA binding buffer PBS containing 1 mM MgCl2 (...,MgCl,PBS/phosphate buffers,7.4,"Truncation: ""Based on the predicted secondary ...","After characterization in the model system, th...",None,NaN
1493,ssDNA,GAATTCCGCGTGTGCACACGCTCACAGTTACTATCGCTACGTTCTT...,64,0.515625,Not reported,NaN,5'-TAATACGACTCACTATAGGGCGAATTCCGCGTGTGC-49N-GT...,49.0,"RNA binding buffer 20 mM Na PO4, pH 7.4/120 mM...",NaN,PBS/phosphate buffers,7.4,NaN,DNA library/pool was used as a template to gen...,None,NaN


In [59]:
df = df[df['pool_random_region'].notna() & df['ph'].notna() & df['kd_(nm)'].notna()]

In [72]:
df.isna().sum()

type_of_nucleic_acid                          0
aptamer_sequence                              0
sequence_length                               0
gc_content                                    0
affinity                                      0
kd_(nm)                                     337
pool_type                                     0
pool_random_region                            0
binding_buffer/conditions                     0
divalent_salt                               442
type_of_the_buffer                            3
ph                                            0
post-selex_modifications_to_the_aptamer     173
additional_information                      798
smiles                                     1263
new_sequence_of_the_target_protein          821
dtype: int64

In [73]:
for i, cond in enumerate(str(df["divalent_salt"])):
    new_text = str(cond)    
    if "CaCl" in cond:
        new_text = cond.replace("CaCl", "CaCl2")    
        if "MgCl" in cond:
            new_text = cond.replace("MgCl", "MgCl2")    
            df.loc[i, "divalent_salt"] = new_text

In [74]:
df['divalent_salt']

0       MgCl
1       MgCl
4       MgCl
5       MgCl
6        NaN
        ... 
1490    MgCl
1491    MgCl
1492    MgCl
1493     NaN
1494     NaN
Name: divalent_salt, Length: 1350, dtype: object

In [75]:
df.isna().sum()

type_of_nucleic_acid                          0
aptamer_sequence                              0
sequence_length                               0
gc_content                                    0
affinity                                      0
kd_(nm)                                     337
pool_type                                     0
pool_random_region                            0
binding_buffer/conditions                     0
divalent_salt                               442
type_of_the_buffer                            3
ph                                            0
post-selex_modifications_to_the_aptamer     173
additional_information                      798
smiles                                     1263
new_sequence_of_the_target_protein          821
dtype: int64

In [80]:
df['divalent_salt'] = df['divalent_salt'].apply(lambda x: 'None' if pd.isna(x) else x)

In [82]:
df['additional_information'] = df['additional_information'].apply(lambda x: 'None' if pd.isna(x) else x)

In [87]:
fixes = { " 1×": "",
    ", and": ",",    " and ": ", ",
    "P04": "PO4",    "H,": "H2",
    "C1": "Cl",    "Mgz+": "Mg2+",
    "CL": "Cl",    "++": "2+",
    "contain-ing 20 mM sodium acetate, 140 mM potassium acetate,and 3 mM magnesium acetate": "20 mM NaOAc, 140 mM KOAc, 3 mM Mg(OAc)2",    " of ": ", ",
    "cl": "Cl",    "c1": "Cl",
    "MgCl3": "MgCl2",    "mMNa2HPO4": "mM Na2HPO4",
    "Cl,,": "Cl2,",    "CI": "Cl",
    "Na PO4": "Na3PO4",    "Phosphate-buffered saline (PBS =": "PBS (",
    "PBS = ": "PBS ",    "Hepes": "HEPES",
    "TrisHCl": "Tris-HCl",    "Tris x Cl": "Tris-HCl",
    "—": "-",    ";": ",",
    "m~": "mM",    "          500 ul, ": "",
    "mMKCl": "mM KCl",    "mM, ": "mM ",
}
for i,cond in enumerate((df["binding_buffer/conditions"])):
    new_text = str(cond)
    for error, fix in fixes.items():        
        if error in cond:
            new_text = new_text.replace(error, fix)    
            df.loc[i, "binding_buffer/conditions"] = new_text

In [88]:
df

,type_of_nucleic_acid,aptamer_sequence,sequence_length,gc_content,affinity,kd_(nm),pool_type,pool_random_region,binding_buffer/conditions,divalent_salt,type_of_the_buffer,ph,post-selex_modifications_to_the_aptamer,additional_information,smiles,new_sequence_of_the_target_protein
0,ssRNA,GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAUGU...,132.0,0.560606,Kd~600µM,600000.0,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98.0,"0.6 ml, 0.5 M LiCl, 20 mM Tris-HCl, pH 7.6, 1 ...",MgCl,Tris Buffers,7.6,Not applicable,The aptamer was reported in DNA (include thymi...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
1,ssRNA,GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUCCG...,133.0,0.593985,Kd<100µM,100000.0,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97.0,"0.6 ml, 0.5 M LiCl, 20 mM Tris-HCl, pH 7.6, 1 ...",MgCl,Tris Buffers,7.6,Not applicable,DNA library/pool was used as a template to gen...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
4,ssDNA,GGTTGGTGTGGTTGG,15.0,0.600000,Kd: 25-200 nM,112.5,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60.0,"200 mM KOAc/50 mM Tris-HCl, pH 7.7/10 mM dithi...",MgCl,Tris Buffers,7.4,Not applicable,Presumed minimized variant was found,None,NaN
5,ssDNA,GGTTGG,6.0,0.666667,Kd: 25-200 nM,112.5,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60.0,"200 mM KOAc/50 mM Tris-HCl, pH 7.7/10 mM dithi...",MgCl,Tris Buffers,7.4,Not applicable,Presumed minimized variant was found,None,TSEDHFQPFFNEKTFGAGEADCGLRPLFEKKQVQDQTEKELFESYI...
6,ssRNA,GGGAGCAUCAGACUUUUAAUCUGACAAUCAAGAAUUCCGUUUUCAG...,96.0,0.333333,Kd: 5 nM,5.0,5'-GGGAGCAUCAGACUUUUAAUCUGACAAUCAAG-N32-AUCUAU...,32.0,"PBS 10.1 mM Na2HPO4, 1.8 mM KH2PO4, 137 mM NaC...",None,Tris Buffers,7.7,Not applicable,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dulbecco's phosphate-buffered saline with MgCl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HBSMClT [HBSMC [HEPES buffered saline (pH 7.4)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HBSMClT [HBSMC [HEPES buffered saline (pH 7.4)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4.5 g/liter glucose, 5 mM MgCl2 in Dulbecco's ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df = df[df['kd_(nm)'].notna() & df['post-selex_modifications_to_the_aptamer'].notna()]

In [93]:
df.isna().sum()

type_of_nucleic_acid                         0
aptamer_sequence                             0
sequence_length                              0
gc_content                                   0
affinity                                     0
kd_(nm)                                      0
pool_type                                    0
pool_random_region                           0
binding_buffer/conditions                    0
divalent_salt                                0
type_of_the_buffer                           1
ph                                           0
post-selex_modifications_to_the_aptamer      0
additional_information                       0
smiles                                     842
new_sequence_of_the_target_protein         553
dtype: int64

In [91]:
df

,type_of_nucleic_acid,aptamer_sequence,sequence_length,gc_content,affinity,kd_(nm),pool_type,pool_random_region,binding_buffer/conditions,divalent_salt,type_of_the_buffer,ph,post-selex_modifications_to_the_aptamer,additional_information,smiles,new_sequence_of_the_target_protein
0,ssRNA,GGGAGAAUUCCCGCGGCAGAAGCCCACCUGGCUUUGAACUCUAUGU...,132.0,0.560606,Kd~600µM,600000.000,5'-GGGAGAATTCCCGCGG-N98-CTGCAGCCCGGGAAGCTT-3',98.0,"0.6 ml, 0.5 M LiCl, 20 mM Tris-HCl, pH 7.6, 1 ...",MgCl,Tris Buffers,7.6,Not applicable,The aptamer was reported in DNA (include thymi...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
1,ssRNA,GGGAGAAUUCCCGCGGCGUUGGCCCAGGAUAAUAGGACGAAAUCCG...,133.0,0.593985,Kd<100µM,100000.000,5'-GGGAGAATTCCCGCGG-N97-CTGCAGCCCGGAAGCTT-3',97.0,"0.6 ml, 0.5 M LiCl, 20 mM Tris-HCl, pH 7.6, 1 ...",MgCl,Tris Buffers,7.6,Not applicable,DNA library/pool was used as a template to gen...,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,NaN
4,ssDNA,GGTTGGTGTGGTTGG,15.0,0.600000,Kd: 25-200 nM,112.500,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60.0,"200 mM KOAc/50 mM Tris-HCl, pH 7.7/10 mM dithi...",MgCl,Tris Buffers,7.4,Not applicable,Presumed minimized variant was found,None,NaN
5,ssDNA,GGTTGG,6.0,0.666667,Kd: 25-200 nM,112.500,5'-CGTACGGTCGACGCTAGC-60N-CACGTGGAGCTCGGATCC-3',60.0,"200 mM KOAc/50 mM Tris-HCl, pH 7.7/10 mM dithi...",MgCl,Tris Buffers,7.4,Not applicable,Presumed minimized variant was found,None,TSEDHFQPFFNEKTFGAGEADCGLRPLFEKKQVQDQTEKELFESYI...
6,ssRNA,GGGAGCAUCAGACUUUUAAUCUGACAAUCAAGAAUUCCGUUUUCAG...,96.0,0.333333,Kd: 5 nM,5.000,5'-GGGAGCAUCAGACUUUUAAUCUGACAAUCAAG-N32-AUCUAU...,32.0,"PBS 10.1 mM Na2HPO4, 1.8 mM KH2PO4, 137 mM NaC...",None,Tris Buffers,7.7,Not applicable,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,2'-fluoro-RNA,GGCGCUCCGACCUUAGUCUCUGUGCCGCUAUAAUGCACGGAUUUAA...,93.0,0.526882,Kd: 36 ± 4 nM,36.000,5'-GGCGCTCCGACCTTAGTCTCTG-N62-GAACCGTGTAGCACAG...,62.0,1X DPBS and 5 mM MgCl2,MgCl,PBS/phosphate buffers,7.4,NaN,In vitro selection of anti-EGFR 2′-fluoropyrim...,None,NaN
1478,2'-fluoro-RNA,5′AGCCGCGAGGGGAGGGAUAGGGUAGGGCGCGGCU3′,34.0,0.735294,Kd: 3.49 ± 1.3,3.490,5'-CGGAATTCGTAATACGACTCACTATAGGGACGCGTGGTACC-N...,50.0,"20 mM HEPES (pH 7.0), 150 mM NaCl, 1 mM MgCl2,...",MgCl,Other Buffers,7.0,"Truncation: ""SE15-8 truncated mini RNA aptamer...",The 2′-fluorine-modified RNA (2′ FY-RNA) libra...,None,NaN
1484,ssDNA,ATCTAACTGCTGCGCCGCCGGGAAAATACTGTACGGTTAGA,41.0,0.512195,Kd:0.8 nM,0.800,5'-ATACCAGCTTATTCAATT-52N-AGATAGTAAGTGCAATCT-3',52.0,4.5 g/liter glucose and 5 mM MgCl2 in Dulbecco...,MgCl,PBS/phosphate buffers,7.4,"Truncation: ""An optimized and truncated DNA se...",Pool information and binding buffer obtained f...,None,NaN
1486,ssDNA,GGGAGACAAGAATAAACGCTCAAGCAGTTGATCCTTTGGATACCCT...,72.0,0.513889,kd: 0.135 nM,0.135,5'-GGGAGACAAGAATAAACGCTCAA-25N-TTCGACAGGAGGCTC...,25.0,"0.2 M NaHCO 3 , 0.5 M NaCl, pH 7.4",None,Other Buffers,7.4,NaN,None,None,NaN


In [97]:
df['type_of_the_buffer'].loc[939] = 'PBS/phosphate buffers'

C:\Users\maest\AppData\Local\Temp\ipykernel_11692\1236452655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type_of_the_buffer'].loc[939] = 'PBS/phosphate buffers'


In [98]:
df['type_of_the_buffer'].loc[939]

'PBS/phosphate buffers'

In [99]:
df['smiles'] = df['smiles'].apply(lambda x: 'None' if pd.isna(x) else x)

C:\Users\maest\AppData\Local\Temp\ipykernel_11692\1932707872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smiles'] = df['smiles'].apply(lambda x: 'None' if pd.isna(x) else x)


In [100]:
df['new_sequence_of_the_target_protein'] = df['new_sequence_of_the_target_protein'].apply(lambda x: 'None' if pd.isna(x) else x)

C:\Users\maest\AppData\Local\Temp\ipykernel_11692\3866709755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_sequence_of_the_target_protein'] = df['new_sequence_of_the_target_protein'].apply(lambda x: 'None' if pd.isna(x) else x)


In [101]:
df.isna().sum()

type_of_nucleic_acid                       0
aptamer_sequence                           0
sequence_length                            0
gc_content                                 0
affinity                                   0
kd_(nm)                                    0
pool_type                                  0
pool_random_region                         0
binding_buffer/conditions                  0
divalent_salt                              0
type_of_the_buffer                         0
ph                                         0
post-selex_modifications_to_the_aptamer    0
additional_information                     0
smiles                                     0
new_sequence_of_the_target_protein         0
dtype: int64

In [105]:
df['kd_(nm)'].unique()

array([6.00000e+05, 1.00000e+05, 1.12500e+02, 5.00000e+00, 2.30000e+01,
       3.20000e+00, 3.00000e+00, 8.10000e+00, 5.10000e-01, 1.90000e-01,
       1.30000e-01, 3.20000e-01, 3.70000e+01, 1.14000e+02, 9.30000e+00,
       1.55000e+02, 8.80000e+01, 2.00000e+04, 8.80000e+03, 3.20000e+02,
       8.70000e+01, 1.97000e+04, 8.00000e+02, 4.00000e+02, 1.00000e+03,
       1.00000e+02, 6.00000e+02, 6.20000e+04, 5.60000e+04, 7.00000e+00,
       4.20000e-01, 4.80000e-01, 1.10000e+00, 2.00000e+01, 1.80000e+00,
       1.40000e+00, 6.00000e+03, 1.00000e+00, 4.00000e+00, 8.00000e+00,
       1.10000e+01, 2.00000e+00, 7.00000e+01, 4.30000e+00, 2.00000e+03,
       2.40000e+00, 1.40000e-01, 1.20000e+01, 5.00000e+02, 3.30900e+03,
       1.47000e-01, 4.70000e+01, 1.27000e-01, 9.40000e-02, 7.20000e+01,
       9.30000e-02, 1.38000e-01, 4.90000e+01, 1.29000e-01, 2.75000e+01,
       3.00000e+01, 3.50000e+01, 1.00000e+01, 5.50000e+00, 3.10000e+00,
       1.50000e+03, 1.26000e+02, 9.00000e-01, 4.20000e+01, 5.000

In [108]:
df.to_csv('Datacon.csv', index=False)